In [ ]:
# Open AI a fine-tuné certaines versions de GPT pour intégrer des arguments supplémentaires en input (demande de fonctions)
# et retourner la fonction demadée par l'usager, sous forme de JSON avec des parmètrs pertinents

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
# JSON object avec quelques paramètres
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
# Properties est lui-même un objet, avec deux éléments (location et unité)
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
# Le seul paramètre dont on a vraiment besoin est le localisation
            "required": ["location"],
        },
    }
]

# La fonction est envoyée au LLM, ce sont les éléments "description" qui sont réellement envoyés au LLM
# Attention à ne pas faire des descriptions trop longues dans les fonctions (context window limitée)

In [ ]:
# On va appeler le modèle avec cette fonction

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
import openai

In [ ]:
# On appelle un modèle doté de cette capacité à comprendre les fonctions, et la fonction créée ci-dessus
# On lui envoie le message ci-dessus
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [ ]:
# La réponse est structurée comme un JSON
# Rq : le nombre de "prompt_tokens" est élevé : en effet on envoie les descriptions des fonctions au modèle
print(response)

In [ ]:
response_message = response["choices"][0]["message"]

In [ ]:
response_message

In [ ]:
response_message["content"]

In [ ]:
response_message["function_call"]

In [ ]:
json.loads(response_message["function_call"]["arguments"])

In [ ]:
# json.loads fournit un objet dictionnaire, qui peut être appelé en input de la fonction codée en dur en début de notebook
# En inspectant la structure de la fonction, on apprend les arguments à mettre en input des fonctions que l'on veut créer 
args = json.loads(response_message["function_call"]["arguments"])

In [ ]:
get_current_weather(args)

In [ ]:
# Que se passe-t-il quand le message n'a rien à voir avec la fonction appelée en même temps ?
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
# Pas d'appel de fonction, mais une réponse du LLM ('content')
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [ ]:
print(response)

In [ ]:
# Avec l'option "function_call", on peut forcer l'appel de la fonction
# function_call='auto' par défaut, le modèle repère lui-même s'il y a lieu d'appeler la fonction
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

In [ ]:
# function_call='none', le modèle est forcé à ne pas appeler la  fonction
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
# Ici, le modèle aimerait appeler la fonction, il renvoie les arguments de la fonction dans le 'content'
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
# function_call={'name':...}, le modèle est forcé à appeler la fonction
# L'argument de l'appel de fonction est déconnecté du contenu (lui-même pas fait pour appeler la fonction)
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages.append(response["choices"][0]["message"])

In [ ]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [ ]:
# On crée un message, non plus en tant qu'usager mais en tant que fonction, dont le contenu est au format demandé
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)